In [ ]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import numpy as np
from math import *
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import re

In [ ]:
powercurve = {0:0,0.5:0,1:0,1.5:0,2:0,2.5:0,3:0,
              3.5:49,4:97,4.5:183,5:268,5.5:369,6:470,
         6.5:580,7:722,7.5:900,8:1103,8.5:1315,9:1526,
        9.5:1705,10:1883,10.5:1940,11:1980,11.5:1995,12:2000,
        12.5:2000,13:2000,13.5:2000,14:2000,14.5:2000,15:2000,
        15.5:2000,16:2000,16.5:2000,17:2000,17.5:2000,18:2000,
        18.5:2000,19:2000,19.5:2000,20:2000,20.5:2000,21:2000,
        21.5:2000,22:2000,22.5:0,23:0,23.5:0,24:0,24.5:0,25:0,25.5:0,
             26:0,26.5:0,27:0,27.5:0,28:0,28.5:0,29:0,29.5:0,30:0}
plt.figure(figsize = (12,5),dpi = 1000)
plt.bar(powercurve.keys(),powercurve.values(),label = 'wind speed',width = 0.3)
plt.title("Vestas.V100.2000 Power Curve")
plt.ylabel("Power (kWh)")
plt.xlabel("Wind speed (m/s)")
plt.show()

In [ ]:
curve = {3.5:49,4:97,4.5:183,5:268,5.5:369,6:470,
         6.5:580,7:722,7.5:900,8:1103,8.5:1315,9:1526,
        9.5:1705,10:1883,10.5:1940,11:1980,11.5:1995,12:2000,
        12.5:2000,13:2000,13.5:2000,14:2000,14.5:2000,15:2000,
        15.5:2000,16:2000,16.5:2000,17:2000,17.5:2000,18:2000,
        18.5:2000,19:2000,19.5:2000,20:2000,20.5:2000,21:2000,
        21.5:2000,22:2000}

In [ ]:
temp_sp = xr.open_dataset("temp_sp.nc").to_array()
speed100 = xr.open_dataset("ws100.nc")
speed100 = np.sqrt(speed100.u100**2+speed100.v100**2).values
#se = [0,251,251,480,1688,2177,2424,2896,3632,3880,5800,6048,6046,6296,7008,7256,7984,8224,9192,9432,10400,10888,11136,11360]
mtime = [250,314,1908,2678,3765,5988,6075,7214,8180,9247,10850,11227]

In [ ]:
# lat:47~64 lon:-25 7
area = []
axes = []
nlon = 0; nlat = 0
# top-left to lower-right
for lat in temp_sp.latitude.values:
    nlon = 0
    for lon in temp_sp.longitude.values:
        if  47 <= lat <= 64 and -25 <= lon <= 7:
            r = cos((lat/180)*np.pi)*6371
            c = 2*np.pi*r
            #km * km
            area.append(c/(4*360)*(2*np.pi*6371/360/4))
            axes.append((nlat,nlon))
        nlon += 1
    nlat += 1

lat_lon = []
for lat in temp_sp.latitude.values:
    for lon in temp_sp.longitude.values:  
        if  47 <= lat <= 64 and -25 <= lon <= 7:
            lat_lon.append((lat,lon))



In [ ]:
name = ['1988020106-mslp.nc','1988020906-mslp.nc','1989102812-mslp.nc','1990020118-mslp.nc',
        '1995011715-mslp.nc','1997122412-mslp.nc','1998010409-mslp.nc','1998122618-mslp.nc',
        '2000112512-mslp.nc','2005110721-mslp.nc','2013122706-mslp.nc','2014021209-mslp.nc']

In [ ]:
se = [227,258,290,347,1892,1956,2646,2705,3754,3780,5974,5999,6048,6090,7192,7239,8175,8193,9200,9295,10814,10876,11203,11249]

In [ ]:
overall_e100m = []
overall_turbine = []
overall_offshore = []
overall_onshore = []
date_list = []
date_number = []
dashed_line = []

In [ ]:
# temperature and surface pressure
# lon: -75 ~ 20 lat:80 ~ 35
# unit : K
# This parameter is the temperature of air at 2m above the surface of land, sea or inland waters

# flag labels the index of storm tracks

# 37 ha = 6 turbines
# 1 ha = 6/37 turbine
# 1ha = 0.01km^2
nn = 0 
flag = 0 

total_energy = 0
total_power = 0
energy_index = 0
power_index = 0

# record initial and end index
index_value = 0

for time in range(0,len(se),2):
    date_number.append(index_value)
    
    flag = int(time/2)
    
    mslp = xr.open_dataset(name[flag])
    mslp = mslp.to_array()[0,:,:]/100
    
    temp2m = temp_sp[0,se[time]:se[time+1],:,:]
    # This parameter is the pressure of the atmosphere at the surface of land, sea and inland water
    sp = temp_sp[1,se[time]:se[time+1],:,:]/100

    # decrease at 0.65K/100m
    temp100m = temp2m - 0.65*98/100
    tempsuf = temp2m + 0.65*2/100


    # p(100) = p(surface) * (1-0.0065*delta/T(2m))^5.225
    # p(surface) hPa
    p100m = sp*((1-0.65/tempsuf)**5.225)

    # kg/m^3
    G = 287.058
    # hPa/(J * Kg-1) = 100 J/m^3 /(J*kg-1) = kg/m^3
    density100m = (p100m/(G*temp100m)*100).values
    
    ws100 = speed100[se[time]:se[time+1],:,:]
    
    # from top-right cornor to lower-left cornor
    # calculate energy
    energy100m = []

    onshore = []
    offshore = []
    turbine = []
    e100m = []
    
    
    # energy100m unit J
    for t in range(se[time+1]-se[time]):
        num = 0
        Sum = 0 
        for index in axes:
            e = 1/2*density100m[t,index[0],index[1]]*area[num]*((ws100[t,index[0],index[1]])**2)*(10**6)
            Sum += e
            energy100m.append(e)
            num+=1
            energy_index += 1
            total_energy += e
        e100m.append(Sum/(num)/(10**10))
        overall_e100m.append(Sum/(num)/(10**10))
    # calculate the power from turbine
    
    
    tpower = []
    for t in range(se[time+1]-se[time]):
        num = 0
        Sum = 0
        ons = 0
        ofs = 0
        ons_num = 0
        ofs_num = 0
        for index in axes:
            power_index+=1
            # lower in cut-in or higher than cut-out
            if 50 <= lat_lon[num][0] <=57 and -13 <= lat_lon[num][1] <=-5:
                ons_num += 1
            else:
                ofs_num += 1
            
            if ws100[t,index[0],index[1]] > max(curve.keys()) or ws100[t,index[0],index[1]] < min(curve.keys()):
                tpower.append(0)
                num += 1
                continue
            # kWh
            
            speed = 0
            if ws100[t,index[0],index[1]] < int(ws100[t,index[0],index[1]])+0.25:
                speed = int(ws100[t,index[0],index[1]]) 
            elif ws100[t,index[0],index[1]] > int(ws100[t,index[0],index[1]])+0.75:
                speed = ceil(ws100[t,index[0],index[1]])
            else :
                speed = int(ws100[t,index[0],index[1]])+0.5
            e = curve[speed]
            e = e * (area[num] / 0.01) * (6 /37) * (10**3)
            
            # 1km^2 1 turbine
            #e = e * (area[num]) * (10**3)
            
            tpower.append(e)
            Sum += e
            total_power+=e
            
            
            if 50 <= lat_lon[num][0] <=57 and -13 <= lat_lon[num][1] <=-5:
                ons += e
            else:
                ofs += e
            num += 1
            
        turbine.append(Sum/num/10**10)
        overall_turbine.append(Sum/num/10**10)
        onshore.append(ons/ons_num/10**10)
        overall_onshore.append(ons/ons_num/10**10)
        offshore.append(ofs/ofs_num/10**10)
        overall_offshore.append(ofs/ofs_num/10**10)
    

    
    pro_energy_tpower = np.array(tpower)/np.array(energy100m)
    
    # 1 W = 1J/s
    #print(min(energy100m),max(energy100m))
    # plot
    
    s_t = list(name[int(time/2)][0:-8])
    s_t.insert(4,"-");s_t.insert(7,"-");s_t.insert(10,"T")
    s_t = "".join(s_t)
    
    
    fig = plt.figure(figsize = [13,13],dpi = 450)
    ax = plt.axes(projection = ccrs.Mercator())
    ax.set_extent([-25., 7., 47., 63.])

    
    pc = ax.contourf(temp_sp.longitude.values[200:329], temp_sp.latitude.values[64:133] ,
                     pro_energy_tpower[(mtime[flag]-se[time])*8901:
                                         (mtime[flag]-se[time]+1)*8901].reshape(69,129), 
                     np.arange(0, 0.7 , 0.1),
                     cmap='jet', transform=ccrs.PlateCarree())
    plt.colorbar(pc, shrink=0.7, label='100%')
    
    cp = ax.contour(mslp.lon, mslp.lat, mslp,
                       np.arange(940, 1050, 10), cmap='jet', transform=ccrs.PlateCarree())
    plt.clabel(cp,inline=1,fontsize=12,fmt='%1.0f',inline_spacing=1, colors='black')

    ax.coastlines(resolution = '50m', color ="black",linewidth = 1.)
    #ax.add_feature(cfeature.LAND)
    grid = ax.gridlines(draw_labels=True,linestyle = "--")

    plt.title(s_t)
    
    plt.show()
    
    period = []
    for i in temp_sp.time.values[se[time]:se[time+1]]:
        period.append(str(i)[0:-16])
    
    plt.figure(figsize = (10,6),dpi = 250)
    plt.bar(temp_sp.time.values[se[time]:se[time+1]],e100m,width = 0.1,color = "orange",label = "energy at 100m")
    plt.bar(temp_sp.time.values[se[time]:se[time+1]],turbine,
            width = 0.1,color = "c",label = "power produced by turbines")
    plt.ylabel("Power (unit : $10^{10}$ J)")
    plt.legend(loc = "upper left")

    axis2 = plt.twinx()
    axis2.plot(temp_sp.time.values[se[time]:se[time+1]],
                  offshore,color="green",label = "offshore power")
    axis2.plot(temp_sp.time.values[se[time]:se[time+1]],
                  onshore,color="red",label = "onshore power")
    plt.axvline(temp_sp.time.values[mtime[flag]],color = "black",label = period[mtime[flag]-se[time]],linestyle = "dashed")
    plt.legend(loc = "upper right")
    plt.ylabel("Power (unit : $10^{10}$ J)")
    plt.title(period[0]+" ~ "+period[-1])
    plt.show()
    
    for t in temp_sp.time.values[se[time]:se[time+1]]:
        date_list.append(str(t)[0:13])
    
    index_value += len(temp_sp.time.values[se[time]:se[time+1]])
    #date_number.append(index_value-1)
    
    dashed_line.append(mtime[flag]-se[time]+nn)
    nn += len(temp_sp.time.values[se[time]:se[time+1]]) 
    

In [ ]:
plt.figure(figsize = (13,6),dpi = 800)
plt.bar(date_list,overall_e100m,width = 0.8,label = "100m energy")
plt.bar(date_list,overall_turbine,width = 0.8,label = "turbine output power")
plt.ylabel("Power (unit : $10^{10}$ J)")
plt.legend(loc = "upper left")

plt.xticks(date_number,rotation = 280)

# unit is different
axis2 = plt.twinx()
axis2.plot(date_list,
           ((np.array(overall_turbine)*10**10)/((np.array(store_total_energy)*10**19)/8901))*100,color = "red",
           label = "Grid-scale proportion")
plt.legend(loc = "upper right")
plt.ylabel("Percent (unit:%)")

for i in dashed_line:
    plt.axvline(date_list[i],color = "black",linestyle = "dashed")

plt.show()

In [ ]:
MMSLP_turbine = 0
MMSLP_offshore = 0
MMSLP_onshore = 0
MMSLP_e100m =0
MMSLP_storm = 0
for i in dashed_line:
    MMSLP_turbine += overall_turbine[i]
    MMSLP_onshore += overall_onshore[i]
    MMSLP_offshore += overall_offshore[i]
    MMSLP_e100m += overall_e100m[i]
    MMSLP_storm += store_total_energy[i]
MMSLP_turbine  = MMSLP_turbine/12
MMSLP_onshore = MMSLP_onshore/12
MMSLP_offshore = MMSLP_offshore/12
MMSLP_e100m = MMSLP_e100m/12
MMSLP_storm = MMSLP_storm/12

print(MMSLP_turbine,MMSLP_onshore,MMSLP_offshore,MMSLP_e100m,MMSLP_storm)
print(MMSLP_turbine/MMSLP_e100m)
print(MMSLP_turbine*10**10/(MMSLP_storm*10**19/8091)*100)

In [ ]:
Rest_turbine = 0
Rest_offshore = 0
Rest_onshore = 0
Rest_e100m =0
Rest_storm = 0

for i in range(len(date_list)):
    if i in dashed_line:
        continue
    Rest_turbine += overall_turbine[i]
    Rest_onshore += overall_onshore[i]
    Rest_offshore += overall_offshore[i]
    Rest_e100m += overall_e100m[i]
    Rest_storm += store_total_energy[i]

Rest_turbine  = Rest_turbine/560
Rest_onshore = Rest_onshore/560
Rest_offshore = Rest_offshore/560
Rest_e100m = Rest_e100m/560
Rest_storm = Rest_storm/560

print(Rest_turbine,Rest_onshore,Rest_offshore,Rest_e100m,Rest_storm)
print(Rest_turbine/Rest_e100m)
print(Rest_turbine*10**10/(Rest_storm*10**19/8091)*100)

In [ ]:
# comparision between onshore and offshore
flag = 0 
onshore = []
offshore = []
for time in mtime:
    ws100 = speed100[time,:,:]
    
    # from top-right cornor to lower-left cornor
    # calculate the power from turbine
    
    num = 0
    ons = 0
    ofs = 0
    ons_num = 0
    ofs_num = 0
    for index in axes:
        # lower in cut-in or higher than cut-out
        if 50 <= lat_lon[num][0] <=57 and -13 <= lat_lon[num][1] <=-5:
            ons_num += 1
        else:
            ofs_num += 1
        
        if ws100[index[0],index[1]] > max(curve.keys()) or ws100[index[0],index[1]] < min(curve.keys()):
            num+=1
            continue
            # kWh
            
        speed = 0
        if ws100[index[0],index[1]] < int(ws100[index[0],index[1]])+0.25:
            speed = int(ws100[index[0],index[1]]) 
        elif ws100[index[0],index[1]] > int(ws100[index[0],index[1]])+0.75:
            speed = ceil(ws100[index[0],index[1]])
        else :
            speed = int(ws100[index[0],index[1]])+0.5
        e = curve[speed]
        e = e * (area[num] / 0.01) * (6 /37) * (10**3)
            
            
        if 50 <= lat_lon[num][0] <=57 and -13 <= lat_lon[num][1] <=-5:
            ons += e
        else:
            ofs += e
        num += 1
    ofs = ofs/ofs_num
    ons = ons/ons_num
    onshore.append(ons)
    offshore.append(ofs)
    
    # 1 W = 1J/s
    #print(min(energy100m),max(energy100m))
    # plot
    flag += 1

In [ ]:
onshore = np.array(onshore)
offshore = np.array(offshore)

name = ['1988-02-01T06','1988-02-09T06','1989-10-28T12',
        '1990-02-01T18','1995-01-17T15','1997-12-24T12',
        '1998-01-04T09','1998-12-26T18','2000-11-25T12',
        '2005-11-07T21','2013-12-27T06','2014-02-12T09']

plt.figure(figsize = (8,6),dpi =150)

plt.barh(name, onshore,label = "average onshore power")
plt.barh(name, -offshore,label = "average offshore power")
plt.legend()
plt.show()